# AIR BNB

## Importing Libraries

In [2]:
import json
import requests as r
import pandas as pd
from pandas.io.json import json_normalize


import datetime
from datetime import datetime
import re
from datetime import date  
import time

## User inputs: 

In [3]:
# check in
while True:
    check_in = input("Please enter check-in date in format 'YYYY-MM-DD' ")
    today = date.today()
    d1 = today.strftime("%Y-%m-%d")
    
    if (re.search("([12]\d{3})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])", check_in)) and (d1<check_in):
        break
    else: 
        print('Your check-in date is either not in correct format or in the past')

# Check out
while True:
    check_out = input("Please enter check-out date in format 'YYYY-MM-DD' ") 
    if (re.search("([12]\d{3})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])", check_out)) and (check_in<check_out):
        break
    else: 
        print('Your check-out date is either not in correct format or in the past')
        
# number of adult
while True:
    num_adults = input('Please enter the number of adults: ')
    if ((num_adults.isdigit()) and (int(num_adults)>0)):
        break
    else:
        print('Please enter number only')
        
# number of children
while True:
    num_childs = input('Please enter the number of children: ')
    if num_childs.isdigit():
        break
    else:
        print('Please enter number only')

# number of infants
while True:
    num_infants = input('Please enter the number of infants: ')
    if num_infants.isdigit():
        break
    else:
        print('Please enter number only')

print(f'Please wait, we are getting the information for date from {check_in} to {check_out}, for {num_adults} adults, {num_childs} childs, {num_infants} infants')

Please enter check-in date in format 'YYYY-MM-DD' 2019-10-23
Your check-in date is either not in correct format or in the past
Please enter check-in date in format 'YYYY-MM-DD' 2019-10-28
Please enter check-out date in format 'YYYY-MM-DD' 2019-11-07
Please enter the number of adults: 2
Please enter the number of children: 0
Please enter the number of infants: 0
Please wait, we are getting the information for date from 2019-10-28 to 2019-11-07, for 2 adults, 0 childs, 0 infants


## Scraping using user inputs: 

In [4]:
# Get value for while loop: 
url = f'https://www.airbnb.com/api/v2/explore_tabs?_format=for_explore_search_web&_intents=p1&adults={num_adults}&auto_ib=false&checkin={check_in}&checkout={check_out}&children={num_childs}&currency=EUR&current_tab_id=home_tab&experiences_per_grid=20&fetch_filters=true&guidebooks_per_grid=20&has_zero_guest_treatment=true&infants={num_infants}&is_guided_search=true&is_new_cards_experiment=true&is_standard_search=true&items_offset=18&items_per_grid=18&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&luxury_pre_launch=true&metadata_only=false&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&query=Berlin&query_understanding_enabled=true&refinement_paths%5B%5D=%2Fhomes&s_tag=V3RB4kXz&satori_version=1.1.9&screen_height=821&screen_size=small&screen_width=705&search_type=pagination&section_offset=7&selected_tab_id=home_tab&show_groupings=true&source=mc_search_bar&supports_for_you_v3=true&timezone_offset=120&toddlers=0&version=1.5.8'
response = r.get(url)
result = response.json()
flat_data = json_normalize(result)
flat_data = flat_data.explore_tabs[0]
flat_data1= json_normalize(flat_data)
a = flat_data1['pagination_metadata.has_next_page'][0]

# Looping (Loop through all the pages)
df = pd.DataFrame()
page_number = 0
while a == True: 
    page_number = page_number + 1 
    k = 18*page_number
    url = f'https://www.airbnb.com/api/v2/explore_tabs?_format=for_explore_search_web&_intents=p1&adults={num_adults}&auto_ib=false&checkin={check_in}&checkout={check_out}&children={num_childs}&currency=EUR&current_tab_id=home_tab&experiences_per_grid=20&fetch_filters=true&guidebooks_per_grid=20&has_zero_guest_treatment=true&infants={num_infants}&is_guided_search=true&is_new_cards_experiment=true&is_standard_search=true&items_offset={k}&items_per_grid=18&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&luxury_pre_launch=true&metadata_only=false&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&query=Berlin&query_understanding_enabled=true&refinement_paths%5B%5D=%2Fhomes&s_tag=V3RB4kXz&satori_version=1.1.9&screen_height=821&screen_size=small&screen_width=705&search_type=pagination&section_offset=7&selected_tab_id=home_tab&show_groupings=true&source=mc_search_bar&supports_for_you_v3=true&timezone_offset=120&toddlers=0&version=1.5.8'
    response = r.get(url)
    result = response.json()
    flattened_data = json_normalize(result)
    flattened_data = json_normalize(flattened_data.explore_tabs[0])
    a = flattened_data['pagination_metadata.has_next_page'][0] # taking the first value of the column
    flattened_data = json_normalize(flattened_data.sections[0]) 
    flattened_data = json_normalize(flattened_data.listings[0])
    flattened_data = flattened_data.set_index('listing.id')
    df = df.append(flattened_data)
    time.sleep(10)


## Cleaning columns names and choosing neccesary columns

In [6]:
# Droping columns:cols = list(df.columns)
cols = list(df.columns)
cols = [re.sub(r'listing\.','',i) for i in cols]
cols = [re.sub(r'pricing_quote\.','',i) for i in cols]
cols = [re.sub(r'verified\.','',i) for i in cols]
df.columns=cols
df1 = df[['city','name','neighborhood','min_nights','public_address','room_and_property_type',
          'preview_amenities','bedroom_label','bed_label','bathroom_label','picture.picture',
          'picture_urls','picture_count','rate.amount_formatted','rate_with_service_fee.amount_formatted',
          'price.total.amount_formatted','price_drop_disclaimer','guest_label','person_capacity','avg_rating',
          'reviews_count','badge_secondary_text']]


# Saving the list
while True:
    save_list = input('Do you want to save the list? Y/N ')
    if (re.search("Y", save_list)):
        df1.to_excel('Airbnb.xlsx')
        print('Thank you for using us! The list is now saved on your computer.')
        break
    elif (re.search("N", save_list)):
        confirm = input('Are you sure? Y/N ')
        if (re.search("Y", confirm)):
            print('Thank you for using us!')
        elif (re.search("N", confirm)):
            df1.to_excel('Airbnb.xlsx')
        print('Thank you for using us! The list is now saved on your computer.')  
        break
    else: 
        print('Please type only Y or N')

Do you want to save the list? Y/N Y
Thank you for using us! The list is now saved on your computer.
